
# сравнение функций потерь от сжатого и сырого контекстов.

### TODO
* Отладка. Проблема Сжатый контекст: N/A
* Пересмотреть функцию потерь, чтобы она отражала задачу сжатия более точно (например, использовать потери восстановления или схожести между оригиналом и сжатым контекстом).
* Усложнить архитектуру модели ContextOptimizer, добавив больше слоев или более продвинутые блоки, такие как GRU или LSTM.

#### Основная цель
Обучить отдельную сеть \( O \) (модуль оптимизации контекста), которая динамически изменяет контекст таким образом, чтобы предсказания исходной языковой модели \( M \) оставались точными, несмотря на уменьшение объема информации в контексте.

### Структура и Процесс Обучения

1. **Основная Языковая Модель \( M \):**
   - \( M \) — существующая языковая модель, такая как GPT-2, которая принимает на вход полный контекст и текущий фрагмент и генерирует предсказания.
   - Цель \( M \) — максимальная точность предсказаний на основе полного контекста.

2. **Модуль Оптимизации Контекста \( O \):**
   - Модуль \( O \) используется для сжатия предыстории \( C \) с учетом текущего фрагмента \( S \), чтобы создать оптимизированный контекст \( C' \).
   - Модуль \( O \) может быть реализован через нейронные сети, такие как трансформеры, рекуррентные сети или другие архитектуры, способные выделять ключевые элементы контекста.

3. **Целевая Функция и Метрики:**
   - Основной задачей является минимизация расхождения между предсказаниями модели \( M \) на основе \( C' \) и \( C \). Целевая функция может быть определена как:
     \[
     L = \text{Loss}(M(C', S), M(C, S))
     \]
   - Для измерения качества сжатия можно использовать метрики, такие как среднеквадратичная ошибка (MSE) между эмбеддингами предсказаний, кросс-энтропия, и т.д.
   - Важно также отслеживать сходимость потерь между \( C' \) и \( C \), чтобы убедиться, что \( C' \) минимально влияет на точность предсказаний.

4. **Процесс Обучения:**
   - **Шаг 1:** Из датасета создаются пары \( (C, S, T) \), где \( C \) — предыстория, \( S \) — текущий фрагмент, и \( T \) — целевой текст для предсказания.
   - **Шаг 2:** Модуль \( O \) сжимает контекст \( C \), производя \( C' = O(C, S) \).
   - **Шаг 3:** Модель \( M \) предсказывает на основе \( (C', S) \) и \( (C, S) \). Сравниваются предсказания для оценки потерь.
   - **Шаг 4:** Потери используются для обновления параметров модуля \( O \), минимизируя влияние сжатия на точность.

5. **Анализ и Диагностика:**
   - Постепенно выводим промежуточные результаты для диагностики, включая анализ предсказаний по сжатому и полному контексту.
   - Выполняем оценку примеров, где предсказания на основе сжатого контекста значительно отличаются от предсказаний на основе полного контекста.
   - Используем фильтрацию данных для обучения, например, исключаем случаи, где текущий фрагмент уже достаточен для точного предсказания.

### Рекомендации для Улучшения

- **Итеративная Диагностика:** Важно регулярно проверять, насколько сжатый контекст \( C' \) сохраняет критическую информацию. Диагностика на нескольких примерах может выявить случаи, где \( O \) недооценивает важные части контекста.
- **Контроль за Переобучением:** Следите за тем, чтобы модель \( O \) не начинала генерировать «фантазии» или добавлять избыточные детали в попытках компенсировать потерю информации.
- **Улучшение Сходимости:** Добавьте регуляризацию или дополнительные ограничения на модель \( O \), чтобы предотвратить отклонения в сжатии, которые могут ухудшить точность предсказаний.

### Ожидаемый Результат

- Модуль \( O \) научится выделять только те части контекста, которые критически важны для точных предсказаний, избегая ненужного увеличения объема информации.
- Модель \( M \), используя \( C' \), будет предсказывать с минимальной потерей точности, что укажет на успешное сжатие и оптимизацию контекста. 


### 1. **Подготовка Данных**
   - **Предварительная обработка данных:** Датасет разбивается на последовательности: предыстория (context), текущий фрагмент (current_fragment), и целевой текст (target).
   - **Фильтрация данных:** Удаляются пустые и нерелевантные записи, например, те, где текущий фрагмент не вносит значимого вклада в предсказание.
   - **Предварительная оценка потерь:** Вычисляются потери для различных комбинаций контекста, что позволяет заранее оценить вклад контекста в точность предсказания.

### 2. **Оптимизация Контекста**
   - **Целевая функция:** Используется MSELoss для измерения расхождения между предсказаниями модели и целевыми значениями.
   - **Сходимость потерь:** Основная цель — сходимость потерь сжатого контекста (compressed_loss) к потерям сырого контекста (raw_loss).
   - **Контроль за «додумыванием»:** Введены штрафы за значительное отклонение между потерями сжатого и сырого контекста, чтобы предотвратить генерацию ложных зависимостей.

### 3. **Адаптивная Регуляция Потерь**
   - **Штрафы за расхождения:** Если потери сжатого контекста значительно отличаются от потерь сырого, в потери добавляется штраф, который стимулирует модель уменьшать эти различия.
   - **Адаптивное обучение:** Обучение адаптируется на основе расхождений, улучшая способность модели сжимать контекст, сохраняя при этом точность.

### 4. **Использование Промежуточных Датасетов**
   - **Сохранение промежуточных результатов:** Включение предсказаний модели в промежуточный датасет, чтобы ускорить и упростить анализ, отладку и последующую оптимизацию.
   - **Фильтрация на этапе подготовки:** Отбираются записи, в которых контекст оказывает значительное влияние на предсказания, отбрасывая те, где контекст несущественен.

### 5. **Визуализация и Отладка**
   - **Графики потерь:** Построение графиков с использованием Plotly для отслеживания динамики потерь сжатого и сырого контекстов.
   - **Примеры для анализа:** Регулярный вывод примеров с различными типами предсказаний (сжатый контекст, сырой контекст, текущий фрагмент) для визуального анализа и проверки корректности работы модели.

### 6. **Управление Временем и Эффективностью**
   - **Оценка времени:** Оценка времени выполнения подготовки данных и каждой эпохи обучения на основе первых пяти циклов, что позволяет планировать ресурсозатраты.
   - **Оптимизация кода:** Фокусировка на ключевых этапах, таких как загрузка моделей, настройка параметров и запуск оптимизации с минимальными затратами времени.

### Основные цели:
- Обеспечение качественного сжатия контекста без потери точности.
- Сведение к минимуму расхождения потерь между сжатым и сырым контекстами.
- Избежание нежелательных интерпретаций и «додумывания» контекста моделью.

## Модульная структура

In [1]:
# Вспомогательные классы и методы

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch.nn as nn
import plotly.graph_objects as go
from IPython.display import display
import os
import json


# Подготовка данных
def prepare_data(dataset, max_length=300):
    """
    Функция подготавливает данные из датасета, включая предысторию, текущий фрагмент и целевой фрагмент текста,
    используя смещающееся окно по элементам датасета.

    :param dataset: Датасет с текстами.
    :param max_length: Максимальная длина текущего фрагмента.
    :return: Список кортежей (предыстория, текущий фрагмент, целевой фрагмент).
    """
    prepared_data = []
    buffer = []
    counter = 0
    for example in dataset:
        text = example["text"].strip()

        # Пропускаем пустые записи
        # TODO Убрать отладочную выборку первых 1000 записей
        if len(text) == 0 or counter > 1000:
            continue
        counter += 1
        buffer.append(text)

        # Если буфер содержит три фрагмента, создаем (context, current_fragment, target)
        if len(buffer) == 3:
            context, current_fragment, target = buffer

            # Обрезаем фрагменты по max_length, если необходимо
            context = context[:max_length].strip()
            current_fragment = current_fragment[:max_length].strip()
            target = target[:max_length].strip()

            # Пропускаем записи, если какой-либо из фрагментов пустой
            if all([context, current_fragment, target]):
                prepared_data.append((context, current_fragment, target))

            # Сдвигаем окно
            buffer.pop(0)
    return prepared_data


# Функции для сохранения и загрузки модели
def save_model(model, optimizer, path="context_optimizer.pth"):
    torch.save(
        {
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        },
        path,
    )
    print(f"Model saved to {path}")


def load_model(model, optimizer, path="context_optimizer.pth"):
    if os.path.exists(path):
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        print(f"Model loaded from {path}")
    else:
        print(f"Model file not found at {path}")


# Визуализация с использованием Plotly FigureWidget
def create_loss_plot():
    fig = go.FigureWidget()
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines+markers", name="Raw Context Loss"))
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines+markers", name="Compressed Context Loss"))
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines+markers", name="Fragment Only Loss"))
    fig.update_layout(title="Графики потерь во время обучения", xaxis_title="Эпоха", yaxis_title="Среднее значение потерь", template="plotly_dark")
    display(fig)
    return fig


def update_loss_plot(fig, epoch, raw_loss, compressed_loss, fragment_loss):
    fig.data[0].x += (epoch,)
    fig.data[0].y += (raw_loss,)
    fig.data[1].x += (epoch,)
    fig.data[1].y += (compressed_loss,)
    fig.data[2].x += (epoch,)
    fig.data[2].y += (fragment_loss,)
    fig.show()


# Функция для расчета compression ratio
def calculate_compression_ratio(raw_embedding, compressed_embedding):
    """
    Вычисляет отношение нормы сжатого контекста к норме сырого контекста.
    """
    raw_norm = torch.norm(raw_embedding, p=2, dim=1)
    compressed_norm = torch.norm(compressed_embedding, p=2, dim=1)
    return compressed_norm / raw_norm


def predict_with_compressed_context(current_fragment, tokenizer, gpt2_model, device, max_new_tokens=50, num_beams=5):
    # Ensure the tokenizer is correctly initialized
    if isinstance(tokenizer, GPT2Tokenizer):
        # Set the pad_token to eos_token if not already set
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token
            # Alternatively, add a new padding token if needed:
            # tokenizer.add_special_tokens({'pad_token': '[PAD]'})

        # Tokenize the current fragment
        inputs = tokenizer(current_fragment, return_tensors="pt", padding=True).to(device)
        input_ids = inputs.get("input_ids")
        attention_mask = inputs.get("attention_mask")  # Adding attention_mask

        # Check that input_ids is not empty
        if input_ids is not None and input_ids.shape[1] > 0:
            with torch.no_grad():
                # Generate predictions directly from current_fragment
                outputs = gpt2_model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,  # Use the attention mask
                    max_new_tokens=max_new_tokens,
                    num_beams=num_beams,
                    early_stopping=True,
                    pad_token_id=tokenizer.pad_token_id,  # Ensure pad_token_id is set correctly
                )
                predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                return predicted_text
        else:
            print(f"Invalid inputs: input_ids length {input_ids.shape[1] if input_ids is not None else 'None'}")
    else:
        print("Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.")

    return "N/A"


# Использование предсказаний из предподготовленных данных
def evaluate_with_prepared_data(prepared_data, model, tokenizer, gpt2_model, device):
    """
    Выполняет оценку качества предсказаний с использованием подготовленных данных.
    """
    for entry in prepared_data:
        context = entry["context"]
        current_fragment = entry["current_fragment"]
        target = entry["target"]
        raw_prediction = entry["raw_prediction"]
        fragment_prediction = entry["fragment_prediction"]

        # Сжатие контекста и предсказание с ним
        compressed_context = model.compress(context)  # Предположим, у нас есть метод compress в модели
        compressed_prediction = predict_with_compressed_context(compressed_context, current_fragment, tokenizer, gpt2_model, device)

        # Сравнение предсказаний
        print(f"Контекст: {context}")
        print(f"Текущий фрагмент: {current_fragment}")
        print(f"Целевой текст: {target}")
        print(f"Предсказание по сырому контексту: {raw_prediction}")
        print(f"Предсказание по текущему фрагменту: {fragment_prediction}")
        print(f"Предсказание по сжатому контексту: {compressed_prediction}\n")


def load_intermediate_dataset(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

In [2]:
# Модель сжатия контекста


class ContextOptimizer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ContextOptimizer, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, input_size)
        self.relu = nn.ReLU()

    def forward(self, context):
        x = self.relu(self.fc1(context))
        x = self.fc2(x)
        return x


# Функция для сжатия контекста
def compress_context(context, optimizer, tokenizer, gpt2_model, device):
    # Здесь выполняется сжатие контекста

    inputs = tokenizer(context, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = gpt2_model(**inputs, output_hidden_states=True)
        raw_context_embedding = outputs.hidden_states[-1].mean(dim=1)

    # Дополнительная проверка перед сжатием
    if raw_context_embedding is None or raw_context_embedding.size(0) == 0:
        print("Warning: raw_context_embedding is empty or invalid")

    # Пример сжатия (эта часть зависит от конкретной реализации оптимизатора)
    compressed_context = optimizer(raw_context_embedding)

    # Проверка формы и значений сжатого контекста
    if not (compressed_context is not None and isinstance(compressed_context, torch.Tensor)):
        print("Compressed context is invalid")

    return compressed_context, raw_context_embedding

In [3]:
import time
import torch.optim as optim

from torch.nn.functional import cosine_similarity


def custom_loss(compressed_output, target_embedding, raw_context_embedding):
    """
    Функция потерь, которая учитывает схожесть с оригиналом и целевым вектором.
    """
    # Потеря восстановления: сравниваем сжатый контекст с исходным контекстом
    reconstruction_loss = 1 - cosine_similarity(compressed_output, raw_context_embedding, dim=1).mean()

    # Потеря предсказания: сравниваем предсказанный выход с целевым
    prediction_loss = nn.MSELoss()(compressed_output, target_embedding)

    # Итоговая потеря: комбинация восстановления и предсказания
    total_loss = prediction_loss + reconstruction_loss
    return total_loss


def train_context_optimizer(optimizer, intermediate_data, tokenizer, gpt2_model, epochs=4, device="cpu"):
    criterion = nn.MSELoss()
    opt = optim.Adam(optimizer.parameters(), lr=0.001)
    raw_loss_values = []
    compressed_loss_values = []
    fragment_loss_values = []

    # Инициализация графика потерь
    loss_fig = create_loss_plot()
    # Для отслеживания потерь по каждому примеру на каждой эпохе
    example_loss_history = {i: [] for i in range(len(intermediate_data))}
    for epoch in range(epochs):
        examples = []
        total_compression_ratio = 0
        total_compressed_loss = 0
        total_divergence = 0  # Переменная для отслеживания расхождения между потерями
        count = 0

        epoch_losses = []  # Для хранения потерь текущей эпохи
        examples_calculated = []
        for idx, entry in enumerate(intermediate_data):
            context = entry["context"]
            current_fragment = entry["current_fragment"]
            target = entry["target"]

            # Используем предвычисленные значения потерь для сырого контекста и текущего фрагмента
            raw_loss = entry["raw_loss"]
            fragment_loss = entry["fragment_loss"]

            # Подготовка целевого предсказания
            target_input = tokenizer(target, return_tensors="pt").to(device)
            with torch.no_grad():
                target_output = gpt2_model(**target_input, output_hidden_states=True)
                target_embedding = target_output.hidden_states[-1].mean(dim=1)

            # Сжатие контекста и вычисление потерь для сжатого контекста в комбинации с текущим фрагментом
            compressed_context, raw_context_embedding = compress_context(context, optimizer, tokenizer, gpt2_model, device)
            compressed_inputs = tokenizer(current_fragment, return_tensors="pt").to(device)
            compressed_context_repeated = compressed_context.unsqueeze(1).repeat(1, compressed_inputs["input_ids"].size(1), 1)
            compressed_inputs_embeds = compressed_context_repeated + gpt2_model.transformer.wte(compressed_inputs["input_ids"])
            compressed_outputs = gpt2_model(inputs_embeds=compressed_inputs_embeds, output_hidden_states=True).hidden_states[-1].mean(dim=1)
            # compressed_loss = criterion(compressed_outputs, target_embedding)
            # Используем custom_loss вместо обычной MSE
            compressed_loss = custom_loss(compressed_outputs, target_embedding, raw_context_embedding)

            # Вычисление расхождения потерь
            divergence = abs(compressed_loss.item() - raw_loss)
            total_divergence += divergence

            # Корректировка потерь для учета расхождения
            adjusted_loss = compressed_loss + divergence  # Добавляем расхождение для управления сходимостью

            # Вычисление compression ratio
            compression_ratio = calculate_compression_ratio(raw_context_embedding, compressed_context)
            total_compression_ratio += compression_ratio.item()

            # Обновление оптимизатора только по скорректированным потерям
            opt.zero_grad()
            adjusted_loss.backward()
            opt.step()

            total_compressed_loss += compressed_loss.item()
            count += 1

            # Сохраняем текущую потерю для анализа изменения потерь
            epoch_losses.append(compressed_loss.item())
            example_loss_history[idx].append(compressed_loss.item())
            # Сохранение примеров для диагностики
            with torch.no_grad():
                compressed_pred = predict_with_compressed_context(compressed_context, current_fragment, tokenizer, gpt2_model, device)
                raw_pred = entry["raw_prediction"]
                fragment_pred = entry["fragment_prediction"]

                # Проверяем и обрабатываем compressed_context корректно перед декодированием
                if compressed_context is not None and isinstance(compressed_context, torch.Tensor):
                    # Пытаемся преобразовать в список токенов, исключая None
                    compressed_tokens = [token for token in compressed_context.squeeze().tolist() if isinstance(token, (int, float)) and 0 <= token < tokenizer.vocab_size]

                    if compressed_tokens:
                        # Декодируем только корректные значения токенов
                        compressed_context_text = tokenizer.decode(compressed_tokens)
                    else:
                        compressed_context_text = "Empty or Invalid tokens found in compressed context"
                else:
                    compressed_context_text = "N/A"

                # Сохраняем примеры в отдельный список для всех примеров
                examples_calculated.append(
                    {
                        "context": context,
                        "compressed_context": compressed_context_text,
                        "current_fragment": current_fragment,
                        "compressed_prediction": compressed_pred,
                        "raw_prediction": raw_pred,
                        "fragment_prediction": fragment_pred,
                        "target": tokenizer.decode(target_input["input_ids"].squeeze().tolist()),
                    }
                )

        # Анализ потерь между эпохами для вывода примеров с максимальным снижением потерь
        if epoch > 0:
            loss_reduction = [
                (i, example_loss_history[i][-2] - example_loss_history[i][-1], examples_calculated[i]["compressed_context"], examples_calculated[i]["compressed_prediction"])
                for i in range(len(epoch_losses))
                if len(example_loss_history[i]) > 1
            ]
            loss_reduction.sort(key=lambda x: x[1], reverse=True)
            top_examples = loss_reduction[:3]  # Три примера с максимальным снижением потерь
        else:
            top_examples = [
                (i, 0, examples_calculated[i]["compressed_context"], examples_calculated[i]["compressed_prediction"]) for i in range(min(3, len(examples_calculated)))
            ]  # Первые три примера для первой эпохи

        avg_compressed_loss = total_compressed_loss / count if count > 0 else 0
        avg_compression_ratio = total_compression_ratio / count if count > 0 else 0
        avg_divergence = total_divergence / count if count > 0 else 0
        raw_loss_values.append(raw_loss)
        compressed_loss_values.append(avg_compressed_loss)
        fragment_loss_values.append(fragment_loss)

        print(f"Epoch {epoch+1}, Avg Compressed Context Loss: {avg_compressed_loss:.4f}, Avg Compression Ratio: {avg_compression_ratio:.4f}, Avg Divergence: {avg_divergence:.4f}")

        # Обновление графика после каждой эпохи
        update_loss_plot(loss_fig, epoch + 1, raw_loss, avg_compressed_loss, fragment_loss)

        # Вывод примеров с максимальным снижением потерь
        print(f"\n=== Примеры с максимальным снижением потерь после эпохи {epoch+1} ===")
        for idx, reduction, compressed_context, compressed_prediction in top_examples:
            example = intermediate_data[idx]
            print(f"\nПример {idx+1} (Снижение потерь: {reduction:.4f}):")
            print(f"Предыстория: {example['context']}")
            print(f"Сжатый контекст: {compressed_context}")  # Используем сохраненный сжатый контекст
            print(f"Текущий фрагмент: {example['current_fragment']}")
            print(f"Предсказание по сжатому контексту: {compressed_prediction}")  # Используем сохраненное предсказание по сжатому контексту
            print(f"Предсказание по сырому контексту: {example['raw_prediction']}")
            print(f"Предсказание по текущему фрагменту: {example['fragment_prediction']}")
            print(f"Целевой текст: {example['target']}")

    return raw_loss_values, compressed_loss_values, fragment_loss_values

In [4]:
import torch.optim as optim
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Настройки устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Загрузка модели GPT-2 и токенизатора
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Загрузка датасета и подготовка промежуточных данных
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
intermediate_dataset_path = "intermediate_dataset.json"

# Генерация промежуточного датасета, если он еще не создан
if not os.path.exists(intermediate_dataset_path):
    generate_intermediate_dataset(dataset, tokenizer, gpt2_model, device=device, intermediate_dataset_path=intermediate_dataset_path)
else:
    # Загрузка промежуточного датасета
    intermediate_data = load_intermediate_dataset(intermediate_dataset_path)

# Инициализация модели сжатия контекста
input_size = 768  # Размер входных данных модели GPT-2
hidden_size = 256  # Скрытый размер для оптимизатора контекста
context_optimizer = ContextOptimizer(input_size, hidden_size).to(device)

# Загрузка сохраненной модели, если она существует
load_model(context_optimizer, context_optimizer, "context_optimizer.pth")

# Запуск обучения
raw_loss_values, compressed_loss_values, fragment_loss_values = train_context_optimizer(context_optimizer, intermediate_data, tokenizer, gpt2_model, epochs=10, device=device)

# Сохранение модели после обучения
save_model(context_optimizer, context_optimizer, "context_optimizer.pth")

Using device: cpu


c:\git\MUIV\concl\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\inimatic\AppData\Local\Temp\ipykernel_16092\595258241.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be all

Model loaded from context_optimizer.pth


FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Raw Context Loss',
              'type': 'scatter',
              'uid': '0f906216-7738-4554-9d52-e42f2ee008b4',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'name': 'Compressed Context Loss',
              'type': 'scatter',
              'uid': 'e3a68387-bf89-4b08-845d-e17762bfbe62',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'name': 'Fragment Only Loss',
              'type': 'scatter',
              'uid': '6291cc5d-116b-4854-b96f-f251a1f91a63',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Графики потерь во время обучения'},
               'xaxis': {'title': {'text': 'Эпоха'}},
               'yaxis': {'title': {'text': 'Среднее значение потерь'}}}
})

Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.
Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.
Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.
Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.
Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.
Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.
Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.
Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.
Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.
Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.
Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.
Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.
Invalid tokenizer object. Ensure the tok

KeyboardInterrupt: 

## Предподготовка данных

In [ ]:
import json
import torch
import time
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
import torch.nn as nn


def generate_intermediate_dataset(dataset, tokenizer, gpt2_model, device="cpu", intermediate_dataset_path="intermediate_dataset.json"):
    intermediate_data = []
    criterion = nn.MSELoss()

    # Подготавливаем данные
    print("Начало подготовки данных...")
    prepared_data = prepare_data(dataset, max_length=500)

    # Оценка времени выполнения цикла для первых пяти итераций
    cycle_times = []
    for i, (context, current_fragment, target) in enumerate(prepared_data[:5]):
        if context is None or current_fragment is None:
            continue

        cycle_start_time = time.time()

        # Подготовка целевого предсказания
        target_input = tokenizer(target, return_tensors="pt").to(device)
        with torch.no_grad():
            target_output = gpt2_model(**target_input, output_hidden_states=True)
            target_embedding = target_output.hidden_states[-1].mean(dim=1)

        # Предсказание по сырому контексту (предыстория + текущий фрагмент)
        raw_inputs = tokenizer(context + " " + current_fragment, return_tensors="pt").to(device)
        with torch.no_grad():
            raw_outputs = gpt2_model(**raw_inputs, output_hidden_states=True)
            raw_combined_embedding = raw_outputs.hidden_states[-1].mean(dim=1)
            raw_loss = criterion(raw_combined_embedding, target_embedding).item()
            raw_prediction = tokenizer.decode(raw_outputs.logits.argmax(-1).squeeze().tolist())

        # Предсказание только по текущему фрагменту
        fragment_input = tokenizer(current_fragment, return_tensors="pt").to(device)
        with torch.no_grad():
            fragment_output = gpt2_model(**fragment_input, output_hidden_states=True)
            fragment_embedding = fragment_output.hidden_states[-1].mean(dim=1)
            fragment_loss = criterion(fragment_embedding, target_embedding).item()
            fragment_prediction = tokenizer.decode(fragment_output.logits.argmax(-1).squeeze().tolist())

        cycle_end_time = time.time()
        cycle_times.append(cycle_end_time - cycle_start_time)

        # Фильтрация по критерию fragment_loss > raw_loss
        if fragment_loss > raw_loss and fragment_loss < 1:
            # Сохранение всех значений и предсказаний
            intermediate_data.append(
                {
                    "context": context,
                    "current_fragment": current_fragment,
                    "target": tokenizer.decode(target_input["input_ids"].squeeze().tolist()),
                    "raw_loss": raw_loss,
                    "fragment_loss": fragment_loss,
                    "raw_prediction": raw_prediction,
                    "fragment_prediction": fragment_prediction,
                }
            )

    # Оценка общего времени выполнения цикла на основе первых пяти измерений
    avg_cycle_time = sum(cycle_times) / len(cycle_times) if cycle_times else 0
    estimated_total_time = avg_cycle_time * len(prepared_data)
    print(f"Оценка времени выполнения цикла: {estimated_total_time:.2f} секунд для {len(prepared_data)} записей.")

    # Продолжение обработки остальных данных после оценки времени
    for i, (context, current_fragment, target) in enumerate(prepared_data[5:]):
        if context is None or current_fragment is None:
            continue

        # Подготовка целевого предсказания
        target_input = tokenizer(target, return_tensors="pt").to(device)
        with torch.no_grad():
            target_output = gpt2_model(**target_input, output_hidden_states=True)
            target_embedding = target_output.hidden_states[-1].mean(dim=1)

        # Предсказание по сырому контексту (предыстория + текущий фрагмент)
        raw_inputs = tokenizer(context + " " + current_fragment, return_tensors="pt").to(device)
        with torch.no_grad():
            raw_outputs = gpt2_model(**raw_inputs, output_hidden_states=True)
            raw_combined_embedding = raw_outputs.hidden_states[-1].mean(dim=1)
            raw_loss = criterion(raw_combined_embedding, target_embedding).item()
            raw_prediction = tokenizer.decode(raw_outputs.logits.argmax(-1).squeeze().tolist())

        # Предсказание только по текущему фрагменту
        fragment_input = tokenizer(current_fragment, return_tensors="pt").to(device)
        with torch.no_grad():
            fragment_output = gpt2_model(**fragment_input, output_hidden_states=True)
            fragment_embedding = fragment_output.hidden_states[-1].mean(dim=1)
            fragment_loss = criterion(fragment_embedding, target_embedding).item()
            fragment_prediction = tokenizer.decode(fragment_output.logits.argmax(-1).squeeze().tolist())

        # Фильтрация по критерию fragment_loss > raw_loss and fragment_loss <= 1
        if fragment_loss > raw_loss and fragment_loss < 1:
            # Сохранение всех значений и предсказаний
            intermediate_data.append(
                {
                    "context": context,
                    "current_fragment": current_fragment,
                    "target": tokenizer.decode(target_input["input_ids"].squeeze().tolist()),
                    "raw_loss": raw_loss,
                    "fragment_loss": fragment_loss,
                    "raw_prediction": raw_prediction,
                    "fragment_prediction": fragment_prediction,
                }
            )

    # Сохранение промежуточного датасета в JSON
    with open(intermediate_dataset_path, "w") as f:
        json.dump(intermediate_data, f, indent=4)

    print(f"Промежуточный датасет создан с {len(intermediate_data)} записями, включая значения потерь и предсказания.")


# Настройки
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Загружаем датасет wikitext-2-raw-v1
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# Инициализация токенизатора и модели GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# Задаем путь для сохранения промежуточного датасета
intermediate_dataset_path = "intermediate_dataset.json"

# Генерация промежуточного датасета
generate_intermediate_dataset(dataset, tokenizer, gpt2_model, device, intermediate_dataset_path)

Using device: cpu
Начало подготовки данных...
Оценка времени выполнения цикла: 435.63 секунд для 999 записей.
Промежуточный датасет создан с 197 записями, включая значения потерь и предсказания.


In [ ]:
# Устарело
import json
import torch
import time
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
import torch.nn as nn


def generate_intermediate_dataset(dataset, tokenizer, gpt2_model, device="cpu", intermediate_dataset_path="intermediate_dataset.json"):
    intermediate_data = []
    criterion = nn.MSELoss()

    # Подготовка данных
    print("Начало подготовки данных...")
    start_time = time.time()
    prepared_data = prepare_data(dataset, max_length=500)
    print("Данные подгружены.")
    # Оценка времени подготовки данных
    preparation_times = []
    for i, (context, current_fragment, target) in enumerate(prepared_data[:5]):
        if context is None or current_fragment is None:
            continue
        preparation_times.append(time.time() - start_time)
        start_time = time.time()

    avg_preparation_time = sum(preparation_times) / len(preparation_times)
    estimated_total_time = avg_preparation_time * len(prepared_data)
    print(f"Оценка времени подготовки данных: {estimated_total_time:.2f} секунд для {len(prepared_data)} записей.")

    for i, (context, current_fragment, target) in enumerate(prepared_data):
        if context is None or current_fragment is None:
            continue

        # Подготовка целевого предсказания
        target_input = tokenizer(target, return_tensors="pt").to(device)
        with torch.no_grad():
            target_output = gpt2_model(**target_input, output_hidden_states=True)
            target_embedding = target_output.hidden_states[-1].mean(dim=1)

        # Предсказание по сырому контексту (предыстория + текущий фрагмент)
        raw_inputs = tokenizer(context + " " + current_fragment, return_tensors="pt").to(device)
        with torch.no_grad():
            raw_outputs = gpt2_model(**raw_inputs, output_hidden_states=True)
            raw_combined_embedding = raw_outputs.hidden_states[-1].mean(dim=1)
            raw_loss = criterion(raw_combined_embedding, target_embedding).item()

        # Предсказание только по текущему фрагменту
        fragment_input = tokenizer(current_fragment, return_tensors="pt").to(device)
        with torch.no_grad():
            fragment_output = gpt2_model(**fragment_input, output_hidden_states=True)
            fragment_embedding = fragment_output.hidden_states[-1].mean(dim=1)
            fragment_loss = criterion(fragment_embedding, target_embedding).item()

        # Сохранение всех значений и предсказаний
        intermediate_data.append(
            {
                "context": context,
                "current_fragment": current_fragment,
                "target": tokenizer.decode(target_input["input_ids"].squeeze().tolist()),
                "raw_loss": raw_loss,
                "fragment_loss": fragment_loss,
            }
        )

    # Сохранение промежуточного датасета в JSON
    with open(intermediate_dataset_path, "w") as f:
        json.dump(intermediate_data, f, indent=4)

    print(f"Промежуточный датасет создан с {len(intermediate_data)} записями, включая значения потерь.")


# Настройки
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Загружаем датасет wikitext-2-raw-v1
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# Инициализация токенизатора и модели GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)  # Используем 'cpu', можно заменить на 'cuda' при использовании GPU

# Задаем путь для сохранения промежуточного датасета
intermediate_dataset_path = "intermediate_dataset.json"

# Генерация промежуточного датасета
generate_intermediate_dataset(dataset, tokenizer, gpt2_model, device, intermediate_dataset_path)

Using device: cpu


c:\git\MUIV\concl\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Начало подготовки данных...
Данные подгружены.
Оценка времени подготовки данных: 276.26 секунд для 999 записей.
Промежуточный датасет создан с 999 записями, включая значения потерь.
